In [1]:
import pandas as pd
import geopandas as gpd
import os
from dash import Dash, html, dcc, Input, Output, State
from folium import folium, Marker, Choropleth, Popup, Icon, LayerControl, GeoJson
import folium
from folium.plugins import HeatMap, MarkerCluster
from branca.element import Template, MacroElement
import plotly.express as px
import numpy as np
from shapely.geometry import Polygon, MultiPolygon
import plotly.graph_objects as go

from general_purpose import add_tourist_spots, add_borders
from district_processing import update_heatmap_graph, update_stats_graph

In [2]:
data_path = os.path.join(os.getcwd(), 'data', 'clean')
listings = pd.read_csv(os.path.join(data_path, "listings_merged.csv"), sep=",")
gdf = gpd.read_file(os.path.join(data_path, 'neighbourhoods.geojson'))

In [ ]:
distritos = list(listings['neighbourhood_group'].unique())

app = Dash(__name__, assets_folder='assets')

app.layout = html.Div(className='container', children=[
	html.H1('Datos de AirBnB en Madrid'),
	
	dcc.Tabs(children=[
		dcc.Tab(label='Información general', children=[
			html.Div(id='general-graphs')
		]),

		dcc.Tab(label='Información por distrito', children=[
			html.H4('Selecciona un distrito:'),
			dcc.Dropdown(
				id='distrito-dropdown-cloropleth',
				options=[{'label': distrito, 'value': distrito} for distrito in distritos],
				value='Centro'
			),
			html.Div(id='distrito-graph-cloropleth', className='map-container')
		])
	])
])


@app.callback(
	Output('distrito-graph-cloropleth', 'children'),
	Input('distrito-dropdown-cloropleth', 'value')
)
def update_cloropleth_graph(distrito):
	df_filtered = listings[listings['neighbourhood_group'] == distrito]
	avg_price_per_neighbourhood = df_filtered.groupby('neighbourhood')['price'].mean().reset_index()
	avg_price_per_neighbourhood = avg_price_per_neighbourhood.merge(gdf, left_on='neighbourhood', right_on='neighbourhood')

	m = folium.Map(
		location=[df_filtered['latitude'].mean(), df_filtered['longitude'].mean()],
		zoom_start=13,
		tiles="CartoDB positron"
	)

	title_html = '''
	<h3 style="font-size:16px; text-align:center; margin-top:10px; margin-bottom:10px;">
		<b>Mapa de cloropletas de: {distrito} (Precio Promedio por Barrio).</b>
	</h3>
	'''.format(distrito=distrito)

	m.get_root().html.add_child(folium.Element(title_html))

	folium.Choropleth(
		geo_data=gdf,
		name='choropleth',
		data=avg_price_per_neighbourhood,
		columns=['neighbourhood', 'price'],
		key_on='feature.properties.neighbourhood',
		fill_color='YlGnBu',
		fill_opacity=0.7,
		line_opacity=0.5,
		legend_name='Precio Promedio (€/noche)',
		highlight=True,
		smooth_factor=0,
	).add_to(m)

	legend_html = '''
		<div style="
			position: fixed;
			bottom: 50px;
			right: 50px;
			width: 150px;
			height: auto;
			background: white;
			border: 2px solid grey;
			z-index: 1000;
			padding: 10px;
			font-size: 14px;
			border-radius: 5px;
			box-shadow: 0 0 10px rgba(0, 0, 0, 0.2);
		">
			<strong>Leyenda</strong><br>
			<i style="background: #ffffcc; width: 20px; height: 20px; display: inline-block;"></i> Bajo<br>
			<i style="background: #a1dab4; width: 20px; height: 20px; display: inline-block;"></i> Medio<br>
			<i style="background: #41b6c4; width: 20px; height: 20px; display: inline-block;"></i> Alto<br>
			<i style="background: #225ea8; width: 20px; height: 20px; display: inline-block;"></i> Muy Alto<br>
		</div>
	'''

	m.get_root().html.add_child(folium.Element(legend_html))

	gdf_with_prices = gdf.merge(avg_price_per_neighbourhood, on="neighbourhood")
	gdf_with_prices["price"] = gdf_with_prices["price"].astype(str).str.replace(',', '.').astype(float)
	gdf_with_prices["price"] = gdf_with_prices["price"].apply(lambda x: f"{x:.2f}")

	gdf_with_prices = gdf_with_prices.rename(columns={'geometry_x': 'geometry'}).set_geometry('geometry')
	if 'geometry_y' in gdf_with_prices.columns:
		gdf_with_prices = gdf_with_prices.drop(columns=['geometry_y'])

	folium.GeoJson(
		data=gdf_with_prices,
		style_function=lambda feature: {
			'fillColor': 'transparent',
			'color': 'black',
			'weight': 0.5,
			'fillOpacity': 0.0,
		},
		popup=folium.GeoJsonPopup(
			fields=['neighbourhood', 'price'],
			aliases=['Distrito:', 'Precio Promedio (€/noche):'],
			localize=True
		)
	).add_to(m)

	map_html = m._repr_html_()

	district_data = update_stats_graph(gdf, distrito, listings)
	district_heatmap = update_heatmap_graph(gdf, distrito, listings)
	add_borders(gdf, m)

	return html.Div([
		district_data,
		html.Iframe(
			srcDoc=map_html,
			style={
				"width": "100%",
				"height": "600px",
				"border": "none"
			}
		),
		district_heatmap
	])

if __name__ == '__main__':
	app.run_server(debug=True)